# Análise Populacional usando PySCF

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) 

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyscf
from pyscf import scf
from pyscf.tools import cubegen # para exportar aquivos cube

## Molécula de H2O

In [ ]:
l0 = 0.9578 # comprimento da ligação O-H em Angstroms
angle = 104.49 * (np.pi / 180) # ângulo H

mol = pyscf.M(
    atom=f'O 0 0 0; H 0 {-l0*np.sin(angle/2)} {l0*np.cos(angle/2)}; H 0. {l0*np.sin(angle/2)} {l0*np.cos(angle/2)}',
    basis = 'aug-cc-pVTZ', unit='Angstrom', verbose=0)

Realizando um cálculo de Hartree-Fock restrito (elétrons emparelhados)

In [ ]:
mf = scf.RHF(mol).to_gpu()
mf.kernel()

print('Total energy', mf.energy_tot())

In [ ]:
from gpu4pyscf.cc import ccsd_incore

mf.with_df = None   # DF CCSD is not supported yet.
mycc = ccsd_incore.CCSD(mf)

mycc.kernel()

print('CCSD total energy', mycc.e_tot)

Energias dos orbitais e cargas de mulliken

In [ ]:
mf.analyze()

Calculando a matriz densidade

In [ ]:
dm = mf.to_cpu().make_rdm1()  # to_cpu

Exportando a densidade eletrônica

In [ ]:
# electron density
cubegen.density(mol, 'water_rho.cube', dm)

Exportando potencial eletrostático

In [ ]:
cubegen.mep(mol, 'water_esp.cube', dm)

## Análise de Bader

Deve instalar o programa de https://theory.cm.utexas.edu/henkelman/code/bader/

In [ ]:
!bader water_rho.cube

In [ ]:
Z = np.array([8,1,1])
N = np.array([9.249086,0.343737,0.343737]).round(3)  # from ACF.dat -> Bader analysis

N.sum()

Q = Z - N
print('Number of electrons from Bader analysis:', N)
print("Bader charges (e):", Q)

## Análise RESP

Baseado em: https://github.com/pyscf/gpu4pyscf/blob/master/examples/22-resp_charge.py

Algoritmo de referência é https://onlinelibrary.wiley.com/doi/epdf/10.1002/qua.26035

In [ ]:
from gpu4pyscf.pop import esp

# ESP charge
q0 = esp.esp_solve(mol, dm)
print('Fitted ESP charge')
print(q0)

In [ ]:
# RESP charge // first stage fitting
q1 = esp.resp_solve(mol, dm)

# Constraints for sum of charges
# sum_constraints = [
#     [c0, [i,j,k,l]],
#     [c1, [i,j,l]]]
#     --> 
#     c0 = q[i] + q[j] + q[k] + q[l]
#     c1 = q[i] + q[j] + q[l]
sum_constraints = [[-q1[0],[1,2]]]  # total charge of H2O is zero

# Constraint for equal charges
# equal_constraints = [
#     [i,j,k],
#     [u,v,w]]
#     -->
#     q[i] = q[j] = q[k] = q[l]
#     q[u] = q[v] = q[w]
equal_constraints = [[1,2]] # H atoms in H2O are equal

# RESP charge // second stage fitting
q2 = esp.resp_solve(mol, dm, resp_a=1e-3,
                    sum_constraints=sum_constraints,
                    equal_constraints=equal_constraints)
print('Fitted partial charge with RESP')
print(q2)

## Análise CHELPG

Baseado em: https://github.com/pyscf/gpu4pyscf/blob/master/examples/15-chelpg.py

In [ ]:
from gpu4pyscf.qmmm import chelpg

q = chelpg.eval_chelpg_layer_gpu(mf)

print('Fitted partial charge with CHELPG')
print(q) 

In [ ]:
# Podemos customizar os raios van der Waals usados no CHELPG
from pyscf.data import radii

q = chelpg.eval_chelpg_layer_gpu(mf, Rvdw=radii.UFF)

print('Fitted partial charge with CHELPG, using UFF radii')
print(q)